In [1]:
import os

In [2]:
%pwd

'C:\\Users\\ahmad\\PycharmProjects\\ISE-Projekt\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\ahmad\\PycharmProjects\\ISE-Projekt'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Classification.constants import *
from Classification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import urllib.request as request
import zipfile
from Classification import logger
from Classification.utils.common import *

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-16 19:04:23,214: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-16 19:04:23,227: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-16 19:04:23,230: INFO: common: created directory at: artifacts]
[2023-12-16 19:04:23,231: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-16 19:04:25,926: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7909994
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d51045470cfb63f35b26cd1c5c8b20ef40254a329e4cfdaa0052fdb91655660c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 48EE:A1C5:2750D46:28FB0CF:657DE6A7
Accept-Ranges: bytes
Date: Sat, 16 Dec 2023 18:04:24 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230028-FRA
X-C

Ende....!!!